Installing Packages

In [ ]:
!pip install nemo_toolkit['all']
!pip install hydra-core==1.1
!pip install evaluate
!pip install jiwer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 144 kB 8.4 MB/s 
  Attempting uninstall: hydra-core
    Found existing installation: hydra-core 1.1.2
    Uninstalling hydra-core-1.1.2:
      Successfully uninstalled hydra-core-1.1.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 362 kB 11.6 MB/s 
     |████████████████████████████████| 212 kB 43.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.4 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=b2cf3e74389f7e4d27784ec77c7bf0fb7487e0824eac680749288cb5c0c8c0e3
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c43677

Importing Libraries

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
import omegaconf
from omegaconf import OmegaConf
from omegaconf import DictConfig
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.plugins import DDPPlugin
import jiwer
import evaluate
from evaluate import load
import sys
import logging

[NeMo W 2022-06-19 05:06:32 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


In [ ]:
sys.argv = ['']
del sys

Defining Dataset Paths

In [ ]:
LANGUAGE = "slices"
path = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset'
manifest_dir = os.path.join(path, LANGUAGE)
train_manifest = f"{manifest_dir}/train/train.json"
dev_manifest = f"{manifest_dir}/val/val.json"
test_manifest = f"{manifest_dir}/test/test.json"

Finetuning Conformer

In [ ]:
@hydra_runner(config_path=r"/content/drive/MyDrive/Colab Notebooks/Sprinklr_Project1/conformer", config_name="conformer_ctc_bpe")
def main(cfg):
    # logging.debug(cfg)
    cfg['model']['train_ds']['manifest_filepath'] = train_manifest
    cfg['model']['validation_ds']['manifest_filepath'] = dev_manifest
    cfg['model']['test_ds']['manifest_filepath'] = test_manifest
    # logging.info(f'Hydra config: {OmegaConf.to_yaml(cfg)}')
    # logging.info("trainer: {}".format(cfg.trainer))
    checkpoint_callback = ModelCheckpoint(dirpath='/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset', 
                                        save_last=True, save_top_k=20,
                                        filename='{epoch}-{val_wer:.2f}-{other_metric:.2f}',
                                        monitor="val_wer", every_n_epochs=10)
    checkpoint_path = None
    trainer = pl.Trainer(gpus=1, max_epochs=100, callbacks=[checkpoint_callback], auto_select_gpus=True)
    asr_model = EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")
    asr_model._wer.log_prediction = True
    asr_model.set_trainer(trainer)
    param_config = DictConfig(cfg['model'])
    asr_model.setup_training_data(param_config.train_ds)
    asr_model.setup_multiple_validation_data(val_data_config=param_config.validation_ds)
    asr_model.setup_multiple_test_data(test_data_config=param_config.test_ds)
    asr_model.spec_augmentation = asr_model.from_config_dict(asr_model.cfg.spec_augment)
    asr_model.setup_optimization(DictConfig(cfg['model']['optim']))
    asr_model.encoder.unfreeze()
    asr_model.decoder.unfreeze()

    trainer.fit(asr_model, ckpt_path=checkpoint_path)
    checkpoint_callback.best_model_path
    checkpoint_callback.best_model_score
    trainer.save_checkpoint

    asr_model.save_to("/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/finetuned_model.nemo")
    if hasattr(cfg.model, 'test_ds') and cfg.model.test_ds.manifest_filepath is not None and False:
        gpu = 1 if cfg.trainer.gpus != 0 else 0
        test_trainer = pl.Trainer(
            gpus=gpu,
            precision=trainer.precision,
            amp_level=trainer.accelerator_connector.amp_level,
            amp_backend=cfg.trainer.get("amp_backend", "native"),
        )
        if asr_model.prepare_test(test_trainer):
            test_trainer.test(asr_model, verbose=True)
if __name__ == '__main__':
    main()

Auto select gpus: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2022-06-18 17:16:57 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.6.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo
[NeMo I 2022-06-18 17:17:08 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-18 17:17:12 mixins:168] Tokenizer SentencePieceTokenizer initialized with 128 tokens
Created a temporary directory at /tmp/tmpi6wrxrz5
Writing /tmp/tmpi6wrxrz5/_remote_module_non_sriptable.py


[NeMo W 2022-06-18 17:17:13 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2022-06-18 17:17:13 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2022-06-18 17:17:13 features:200] PADDING: 0


[NeMo W 2022-06-18 17:17:14 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-06-18 17:17:14 nemo_logging:349] /usr/local/lib/python3.7/dist-packa

...

[NeMo I 2022-06-18 20:39:44 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:39:44 wer_bpe:213] reference:hand rule so now the d space frame element we shown here
[NeMo I 2022-06-18 20:39:44 wer_bpe:214] predicted:cone role so now this the thre d plus for am element is shown her
[NeMo I 2022-06-18 20:39:44 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:39:44 wer_bpe:213] reference:the main memory into a number of
[NeMo I 2022-06-18 20:39:44 wer_bpe:214] predicted:the mein memory into a nmmer
[NeMo I 2022-06-18 20:39:45 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:39:45 wer_bpe:213] reference:enforce the continuity later on so thus we obtained an intermediate value for velocity
[NeMo I 2022-06-18 20:39:45 wer_bpe:214] predicted:inforce the compute it lo so thats we obtained and intermt it value for velocity
[NeMo I 2022-06-18 20:39:45 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:39:45 wer_bpe:213] reference:it for three connected graph first that means if g is a three connected graph and it doesn't contain
[

Validation: 0it [00:00, ?it/s]

[NeMo I 2022-06-18 20:41:45 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:41:45 wer_bpe:213] reference:hand rule so now the d space frame element we shown here
[NeMo I 2022-06-18 20:41:45 wer_bpe:214] predicted:con role so now this the sttheady s plus prom element is shown your
[NeMo I 2022-06-18 20:41:45 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:41:45 wer_bpe:213] reference:the main memory into a number of
[NeMo I 2022-06-18 20:41:45 wer_bpe:214] predicted:in the math memory into a number of
[NeMo I 2022-06-18 20:41:45 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:41:45 wer_bpe:213] reference:enforce the continuity later on so thus we obtained an intermediate value for velocity
[NeMo I 2022-06-18 20:41:45 wer_bpe:214] predicted:inforce a competoryterm so that is we obtained and intermedtiate o value four velocity
[NeMo I 2022-06-18 20:41:46 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:41:46 wer_bpe:213] reference:it for three connected graph first that means if g is a three connected graph and it do

Validation: 0it [00:00, ?it/s]

[NeMo I 2022-06-18 20:43:46 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:43:46 wer_bpe:213] reference:hand rule so now the d space frame element we shown here
[NeMo I 2022-06-18 20:43:46 wer_bpe:214] predicted:ine rule so now this the' thready plate remlement is shown he
[NeMo I 2022-06-18 20:43:46 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:43:46 wer_bpe:213] reference:the main memory into a number of
[NeMo I 2022-06-18 20:43:46 wer_bpe:214] predicted:and the main memory intero nummer
[NeMo I 2022-06-18 20:43:46 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:43:46 wer_bpe:213] reference:enforce the continuity later on so thus we obtained an intermediate value for velocity
[NeMo I 2022-06-18 20:43:46 wer_bpe:214] predicted:inforce the cop utilit w so this we obtained and intermediate value fo velocity
[NeMo I 2022-06-18 20:43:46 wer_bpe:212] 
    
[NeMo I 2022-06-18 20:43:46 wer_bpe:213] reference:it for three connected graph first that means if g is a three connected graph and it doesn't contain
[

Loading Finetuned Model

In [ ]:
finetuned_model = EncDecCTCModelBPE.restore_from("/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/finetuned_model.nemo")

Creating List of Paths

In [ ]:
os.chdir('/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test')
directory = 'wav'
manifest_dir_url = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav'
new_list_of_paths = []

for filename in os.listdir(directory):
    new_list_of_paths.append(f"{manifest_dir_url}/{filename}")
print(new_list_of_paths)

['/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/0002789d96a16b1205ecb7f9b9f4fbe2d8d1fe42be400d8b82070f97.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/0006715b5a8f57f9f6b248fee16b187ee62d4e437907b6009cd07c1e.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/0008f0a4237eaff4ab6577cee78f049625ce3bdb6eac5e9c1f35fd56.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/0003952f37206f7c761442924685b0bd9d49bbebb39b8f0f9bef47dd.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/0004ca5248e478aa0279f2fef8cf247fab34e8d9f41629ab20fac66c.wav', '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test/wav/000020db7e99a1de454a92346534c843f7f712fd7d3160a7d575af4a.wav

Creating List of actual transcripts

In [ ]:
os.chdir('/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/slices/test')
directory = 'wav'
dir2 = '/content/drive/Othercomputers/My MacBook Pro/Google_Drive/Finetuning Conformer Dataset/corrected_txt'
list_of_actual_statements = []

for filename in os.listdir(directory):
    name_of_file = os.path.splitext(filename)[0]
    transcript_file_path = f"{dir2}/{name_of_file}.txt"
    file = open(transcript_file_path, 'r')

    list_of_actual_statements.append(file.read())

    file.close()

In [ ]:
wer = load("wer")

Getting Predictions using Finetuned Model

In [ ]:
finetuned_model_predictions = finetuned_model.transcribe(paths2audio_files=new_list_of_paths)
print(finetuned_model_predictions)

Transcribing:   0%|          | 0/25 [00:00<?, ?it/s]

['tc appear as zero here and at very have frequencies this is going to be an opten and the oleput is', 'tbver the process and this was in the seventeen scturly as you can see and e took', 'instructions are whre', 'p l will get distoged so the output c will be at ze', 'is a drop that iss found it may be a drop it may be like a ing structure depending', 'so at this point this plaus this plus this nn sohw on will give me at the corresponding', 'what we callssum is then created now example is one one zero zero is tweltn this is multiplitation and what', 'like your hardnest on your p c magnatic tp which is primarily used in many locations', 'it called the low temperature process y it is chemical route and y it is part of', 'say this is an a b two type of structure laoww space and this is based on the cuble lattices where', 'so we have round the work bg so many times in one second it velocity is the restance that', 'amounent of dee electin hole that it can a generate the', 'two lolopes but a

Initialising Pretrained Model

In [ ]:
pretrained_model = EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_large")

Getting Predictions using Pretrained Model

In [ ]:
pretrained_model_predictions = pretrained_model.transcribe(paths2audio_files=new_list_of_paths)

Transcribing:   0%|          | 0/25 [00:00<?, ?it/s]

Removing WhiteSpace Characters

In [ ]:
list_of_actual_statements = jiwer.RemoveEmptyStrings()(list_of_actual_statements)
pretrained_model_predictions = jiwer.RemoveEmptyStrings()(pretrained_model_predictions)
finetuned_model_predictions = jiwer.RemoveEmptyStrings()(finetuned_model_predictions)

WER using FineTuned Model on NPTEL Indian English Speech Dataset

In [ ]:
wer_score = jiwer.wer(list_of_actual_statements, finetuned_model_predictions)
print(wer_score)

1.0152801358234296


WER using Pre-trained model on NPTEL Indian English Speech Dataset

In [ ]:
wer_score = jiwer.wer(list_of_actual_statements, pretrained_model_predictions)
print(wer_score)

1.0260328239954726


Downloading Commonvoice English Dataset

In [ ]:
os.chdir("/content/drive/Othercomputers/My MacBook Pro/Google_Drive/CommonVoice English Dataset")

In [ ]:
!wget https://mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com/cv-corpus-5.1-2020-06-22/en.tar.gz/

--2022-06-19 05:11:16--  https://mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com/cv-corpus-5.1-2020-06-22/en.tar.gz/
Resolving mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com (mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com)... 52.92.209.162, 2600:1fa0:408c:411:34da:c059::
Connecting to mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com (mozilla-common-voice-datasets.s3.dualstack.us-west-2.amazonaws.com)|52.92.209.162|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-06-19 05:11:16 ERROR 403: Forbidden.

